In [1]:
%matplotlib inline

In [2]:
import pandas as pd
from psyrun import NpzStore

In [3]:
data = NpzStore().load('../data/memory.npz')
data['dimensions'][np.logical_not(np.isfinite(data['dimensions']))] = -1
data['n_neurons'][np.logical_not(np.isfinite(data['n_neurons']))] = -1

In [4]:
import xarray as xr

In [5]:
data.keys()

dict_keys(['backend', 'model', 'memory', 'n_neurons', 'dimensions', 'trial', 'event_times', 'time'])

In [6]:
np.unique(data['model'])

array(['circ_conv', 'nback'], 
      dtype='<U9')

In [7]:
ds = xr.Dataset({
    'memory': (['idx', 't_idx'], data['memory']),
    'time': (['idx', 't_idx'], data['time']),
    'trial': (['idx'], data['trial']),
    'n_neurons': (['idx'], data['n_neurons']),
    'dimensions': (['idx'], data['dimensions']),
    'backend': (['idx'], data['backend']),
    'model': (['model'], data['model']),
}, coords={'idx': range(len(data['trial'])), 't_idx': range(data['time'].shape[1])})

In [8]:
def get_coords(x):
    coords = np.unique(x)
    try:
        coords[np.logical_not(np.isfinite(coords))] = -1
    except:
        pass
    return np.unique(coords)

keys = ['model', 'backend', 'n_neurons', 'dimensions', 'trial']
coords = {k: get_coords(data[k]) for k in keys}
coords['t_idx'] = range(data['time'].shape[1])
shape = tuple(len(coords[k]) for k in keys) + (data['time'].shape[1],)

In [9]:
ds = xr.Dataset({
        'memory': (keys + ['t_idx'], np.empty(shape)),
        't': (keys + ['t_idx'], np.empty(shape)),
    }, coords=coords)
ds

<xarray.Dataset>
Dimensions:     (backend: 4, dimensions: 2, model: 2, n_neurons: 2, t_idx: 8804, trial: 5)
Coordinates:
  * t_idx       (t_idx) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * backend     (backend) <U9 'ocl_cpu' 'ocl_gpu' 'optimized' 'reference'
  * dimensions  (dimensions) float64 -1.0 500.0
  * model       (model) <U9 'circ_conv' 'nback'
  * n_neurons   (n_neurons) float64 -1.0 500.0
  * trial       (trial) int64 0 1 2 3 4
Data variables:
    memory      (model, backend, n_neurons, dimensions, trial, t_idx) float64 0.0 ...
    t           (model, backend, n_neurons, dimensions, trial, t_idx) float64 0.0 ...

In [10]:
import itertools
for x in itertools.product(*(coords[k] for k in keys)):
    sel = np.ones_like(data['memory'], dtype=bool)
    for i, k in enumerate(keys):
        sel = np.logical_and(sel, (data[k] == x[i])[:, None])
        #print(k, x[i])
        #print(np.any(data[k] == x[i]))
        #print(np.any(sel))
    loc = {k: x[i] for i, k in enumerate(keys)}
    #print(data['memory'].shape, sel.shape, sel)
    if np.any(sel):
        ds['memory'].loc[loc] = data['memory'][sel]
        ds['t'].loc[loc] = data['time'][sel]

In [11]:
def mem_for(benchmark, backend, n_neurons=-1, dimensions=-1):
    da = ds.loc[{
        'model': benchmark,
        'backend': backend,
        'n_neurons': n_neurons,
        'dimensions': dimensions,
    }]
    return np.max(da['memory'] / 1024. / 1024.)

In [12]:
print('reference:', mem_for('circ_conv', 'reference', 500, 500))
print('optimized:', mem_for('circ_conv', 'optimized', 500, 500))

reference: <xarray.DataArray 'memory' ()>
array(321.48828125)
Coordinates:
    backend     <U9 'reference'
    dimensions  float64 500.0
    model       <U9 'circ_conv'
    n_neurons   float64 500.0
optimized: <xarray.DataArray 'memory' ()>
array(346.21484375)
Coordinates:
    backend     <U9 'optimized'
    dimensions  float64 500.0
    model       <U9 'circ_conv'
    n_neurons   float64 500.0


In [13]:
print('reference:', mem_for('nback', 'reference'))
print('optimized:', mem_for('nback', 'optimized'))

reference: <xarray.DataArray 'memory' ()>
array(168.93359375)
Coordinates:
    backend     <U9 'reference'
    dimensions  float64 -1.0
    model       <U9 'nback'
    n_neurons   float64 -1.0
optimized: <xarray.DataArray 'memory' ()>
array(165.8828125)
Coordinates:
    backend     <U9 'optimized'
    dimensions  float64 -1.0
    model       <U9 'nback'
    n_neurons   float64 -1.0
